In [2]:

# Imports and basic setup
import tellurium as te
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# ── Cell 3 ──
# 1) Declare the Antimony model as a multiline string
ant = """
model birth_death()
  // Initial species count
  U = 100
  V = 100
  W = 100
  X = 100
  Y = 100

  // Kinetic parameters
  size = 1000
  W_birth = 0.4
  W_death = 0.1
  Y_birth = 0.9
  Y_death = 0.15
  U_in = 0.1
  U_out = 0.1
  X_in = 0.1
  X_out = 0.1


  // Symbiosis reaction: 
  W_symbiosis:    => W;       W_birth / size * W * Y
  V_symbiosis:    => V;       W_birth / size * V * Y
  Y_symbiosis:    => Y;       Y_birth / size * (W+V) * Y


  // Death reaction:
  V_death_reaction:    V => ;      W_death     * V
  W_death_reaction:    W => ;      W_death     * W
  Y_death_reaction:    Y => ;      Y_death     * Y


  // Quiescence reaction:
  U_quiescence_in:   V => U;     U_in * V
  X_quiescence_in:   W => X;     X_in * W 
  U_quiescence_out:  U => V;     U_out * U
  X_quiescence_out:  X => W;     X_out * X


  // Competition reaction:
  V_compete:     V => ;     W_birth / (size * size) * V * (W + V) * Y
  W_compete:     W => ;     W_birth / (size * size) * (V + W) * W * Y
  Y_compete:     Y => ;     Y_birth / (size * size) * (V + W) * Y * Y

end
"""

# 2) Load into a RoadRunner instance
rr = te.loadAntimonyModel(ant)

# 3) Select Gillespie’s SSA integrator
rr.setIntegrator('gillespie')

# ── Cell 4 ──
# 4) Simulate multiple trajectories
n_replicates = 1000     # number of stochastic realizations
n_points     = 500      # number of intervals; yields n_points+1 timepoints
all_X        = np.empty((n_replicates, n_points+1))
times        = None     # will be filled on first simulation

for i in tqdm(range(n_replicates), desc="Simulating trajectories"):
    rr.resetAll()       # revert to initial conditions
    rr.setSeed(i)       # reproducible RNG seed per replicate
    traj = rr.gillespie(0, 100, n_points)  # simulate from t=0 to t=100
    if times is None:
        times = traj[:, 0]          # time‐vector (common to all runs)
    all_X[i, :] = traj[:, 1]        # store X‐counts

# ── Cell 5 ──
# 5) Compute mean and plot
mean_X = np.mean(all_X, axis=0)

plt.figure(figsize=(6,4))
plt.plot(times, mean_X)
plt.xlabel('Time')
plt.ylabel('Mean $X$')
plt.title('Mean trajectory of $X$ over {} replicates'.format(n_replicates))

# 6) Save figure as PDF
plt.savefig('mean_trajectory.pdf')
plt.show()

# ── Cell 6 ──
# 7) Now you have:
#    * times  : 1D array of shape (n_points+1,)
#    * all_X  : 2D array of shape (n_replicates, n_points+1)
#    * mean_X : 1D array of shape (n_points+1,)
#
# You can further save these to disk, compute variances or histograms, etc.
#
# Example: save arrays in compressed NumPy format
np.savez_compressed('birth_death_trajectories.npz',
                    times=times,
                    all_X=all_X,
                    mean_X=mean_X)

Simulating trajectories:   0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: could not broadcast input array from shape (500,) into shape (501,)

In [6]:

# ── Cell 2: Imports ──
import tellurium as te
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# ── Cell 3: Antimony model + simulation settings ──
ANT_MODEL = """
model birth_death()
  // Initial species count
  U = 100
  V = 100
  W = 100
  X = 100
  Y = 100

  // Kinetic parameters
  size = 1000
  W_birth = 0.4
  W_death = 0.1
  Y_birth = 0.9
  Y_death = 0.15
  U_in = 0.1
  U_out = 0.1
  X_in = 0.1
  X_out = 0.1


  // Symbiosis reaction: 
  W_symbiosis:    => W;       W_birth / size * W * Y
  V_symbiosis:    => V;       W_birth / size * V * Y
  Y_symbiosis:    => Y;       Y_birth / size * (W+V) * Y


  // Death reaction:
  V_death_reaction:    V => ;      W_death     * V
  W_death_reaction:    W => ;      W_death     * W
  Y_death_reaction:    Y => ;      Y_death     * Y


  // Quiescence reaction:
  U_quiescence_in:   V => U;     U_in * V
  X_quiescence_in:   W => X;     X_in * W 
  U_quiescence_out:  U => V;     U_out * U
  X_quiescence_out:  X => W;     X_out * X


  // Competition reaction:
  V_compete:     V => ;     W_birth / (size * size) * V * (W + V) * Y
  W_compete:     W => ;     W_birth / (size * size) * (V + W) * W * Y
  Y_compete:     Y => ;     Y_birth / (size * size) * (V + W) * Y * Y

end
"""
T_END    = 100.0
N_POINTS = 500
N_REPS   = 1000

# Precompute the time‐vector once
_rr0  = te.loadAntimonyModel(ANT_MODEL)
_rr0.setIntegrator('gillespie')
TIMES = _rr0.gillespie(0, T_END, N_POINTS)[:,0]

# ── Cell 4: Top‐level simulation function ──
def simulate_once(seed: int) -> np.ndarray:
    """
    Simulate one Gillespie trajectory of U from the global ANT_MODEL
    using seed 'seed' and return the U‐counts over time.
    """
    rr = te.loadAntimonyModel(ANT_MODEL)
    rr.setIntegrator('gillespie')
    rr.resetAll()
    rr.setSeed(seed)
    traj = rr.gillespie(0, T_END, N_POINTS)
    return traj[:,1]  # column 1 is U

# ── Cell 5: Parallel execution ──
seeds = list(range(N_REPS))
with ProcessPoolExecutor() as executor:
    results_iter = executor.map(simulate_once, seeds)
    all_U = np.vstack(
        list(tqdm(results_iter,
                  total=N_REPS,
                  desc="Simulating trajectories"))
    )
# all_U.shape == (N_REPS, N_POINTS+1)

# ── Cell 6: Post‐processing example ──
mean_U = np.mean(all_U, axis=0)
np.savez_compressed('birth_death_parallel.npz',
                    times=TIMES,
                    all_U=all_U,
                    mean_U=mean_U)
print(f"Completed {N_REPS} trajectories in parallel.")

Process SpawnProcess-21:
Traceback (most recent call last):
  File "/Users/xaverwangerpohl/anaconda3/envs/coevolution/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/xaverwangerpohl/anaconda3/envs/coevolution/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/xaverwangerpohl/anaconda3/envs/coevolution/lib/python3.9/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/xaverwangerpohl/anaconda3/envs/coevolution/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'simulate_once' on <module '__main__' (built-in)>
Process SpawnProcess-22:
Traceback (most recent call last):
  File "/Users/xaverwangerpohl/anaconda3/envs/coevolution/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/xave

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore